In [307]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import pygcransac
from time import time

correspondences = np.loadtxt('img/pose6dscene_points.txt')
gt_pose = np.loadtxt('img/pose6dscene_gt.txt')
intrinsic_params = np.loadtxt('img/pose6dscene.K')

print("Number of original correspondences loaded = ", str(len(correspondences)))

# Adding outliers to the scene
# TODO: replace with an actual example with outliers
outlier_number = round(3.0 * correspondences.shape[0])
mins = np.min(correspondences, axis=0)
maxs = np.max(correspondences, axis=0)

mins[0] = 0
mins[1] = 0

outliers = []
for i in range(outlier_number):
    for dim in range(5):
        outliers.append(random.uniform(mins[dim], maxs[dim]))
outliers = np.array(outliers).reshape(-1, 5)
correspondences = np.concatenate((correspondences, outliers))
print("Number of correspondences with outliers = ", str(len(correspondences)))
print("Outlier ratio = ", outlier_number / correspondences.shape[0])



Number of original correspondences loaded =  95
Number of correspondences with outliers =  380
Outlier ratio =  0.75


In [308]:
def verify_opencv(normalized_corrs, K):
    n = len(normalized_corrs)
    imagePoints = np.float32([normalized_corrs[i][0:2] for i in np.arange(n)]).reshape(-1,2)
    worldPoints = np.float32([normalized_corrs[i][2:5] for i in np.arange(n)]).reshape(-1,3)
    dist_coeffs = np.zeros((4,1))
    camera_matrix = np.identity(3)

    threshold = 2.0
    normalized_threshold = threshold / (K[0, 0] + K[1, 1]) / 2.0;    

    success, rotation_vector, translation_vector, inliers = cv2.solvePnPRansac(
        worldPoints, 
        imagePoints, 
        camera_matrix, 
        dist_coeffs, 
        flags = cv2.SOLVEPNP_ITERATIVE,
        iterationsCount = 1000,
        reprojectionError = normalized_threshold)
    
    mask = np.zeros(n)
    if not success:
        return np.zeros((3, 4)), mask
    mask[inliers] = 1
    
    rotation, _ = cv2.Rodrigues(rotation_vector)
    pose = np.concatenate((rotation, translation_vector), axis=1)
    return pose, mask

def verify_pygcransac(corrs, K):        
    threshold = 2.0
    normalized_threshold = threshold / (K[0, 0] + K[1, 1]) / 2.0;    
    pose, mask = pygcransac.find6DPose(
        np.ascontiguousarray(corrs),
        min_iters = 50,
        max_iters = 1000,
        probabilities = [], # Inlier probabilities. This is not used if the sampler is not 3 (NG-RANSAC) or 4 (AR-Sampler)
        sampler = 0, # Sampler index (0 - Uniform, 1 - PROSAC, 2 - P-NAPSAC, 3 - NG-RANSAC, 4 - AR-Sampler)
        threshold = normalized_threshold,  # Inlier-outlier threshold
        conf = 0.99) # RANSAC confidence
    return pose, mask

def normalize_image_points(corrs, K): 
    n = len(corrs)
    normalized_correspondences = np.zeros((corrs.shape[0], 5))
    inv_K = np.linalg.inv(K)

    for i in range(n):
        p1 = np.append(correspondences[i][0:2], 1)
        p2 = inv_K.dot(p1)
        normalized_correspondences[i][0:2] = p2[0:2]
        normalized_correspondences[i][2:] = correspondences[i][2:]
    return normalized_correspondences

def calculate_error(gt_pose, est_pose):
    
    R2R1 = np.dot(gt_pose[:, 0:3].T, est_pose[:, 0:3])
    cos_angle = max(-1.0, min(1.0, 0.5 * (R2R1.trace() - 1.0)))
    
    err_R = np.arccos(cos_angle) * 180.0 / np.pi
    err_t = np.linalg.norm(gt_pose[:, 3] - est_pose[:, 3])
    
    return err_R, err_t


In [309]:
normalized_correspondences = normalize_image_points(correspondences, intrinsic_params)

t = time()

pose, mask = verify_pygcransac(normalized_correspondences, intrinsic_params)
print (time()-t, ' sec GC-RANSAC')

err_R, err_t = calculate_error(gt_pose, pose)

print('Inlier number = ', np.sum(mask))
print('Rotation error = ', err_R, '°')
print('Translation error = ', err_t, ' mm')

t = time()

# Note that OpenCV dies with this outlier ratio and is much slower too.
pose, mask = verify_opencv(normalized_correspondences, intrinsic_params)
print (time()-t, ' sec OpenCV')

err_R, err_t = calculate_error(gt_pose, pose)

print('Inlier number = ', np.sum(mask))
print ('Rotation error = ', err_R, '°')
print ('Translation error = ', err_t, ' mm')

0.0026285648345947266  sec GC-RANSAC
Inlier number =  95
Rotation error =  2.7612681746424212e-05 °
Translation error =  0.00012228217502662872  mm
0.042549848556518555  sec OpenCV
Inlier number =  0.0
Rotation error =  120.00000000000001 °
Translation error =  917.0814487116129  mm
